ADMM

http://statsmaths.github.io/stat612/lectures/lec23/lecture23.pdf

http://www.stronglyconvex.com/blog/admm-revisited.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### LASSO

Solving the Lasso via ADMM:

https://statweb.stanford.edu/~candes/math301/Lectures/Consensus.pdf

$$ \min_{\mathbf{x}} \frac{1}{2} {\lVert \mathbf{A}\mathbf{x} - \mathbf{b} \rVert}^2_2 + \lambda {\lVert \mathbf{x} \rVert}_1 $$

In [2]:
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import cg, lsqr
from scipy.linalg import norm

In [3]:
def prox_l1(x, tau):
    return np.sign(x) * np.maximum(np.abs(x) - tau, 0)

def lasso_admm(A, b, lmbda, rho=0.1, max_iter=10000):
    """Solves
    0.5*||A*x - y||^2_2 + lmbda*||x||_1
    using ADMM
    """
    x0 = np.random.randn(A.shape[1],)
    n = len(x0)
    x = x0.copy()
    z = x0.copy()
    u = x0.copy()
    
    is_converged = False
    for iteration in range(max_iter):
        # update x
        mv = lambda x: A.rmatvec(A.matvec(x)) + rho*x
        L = LinearOperator((n,n), matvec=mv)
        x = cg(L, A.rmatvec(b) + rho*(z - u))[0]
        # update z
        z = prox_l1(x + u, lmbda/rho)
        # update u
        r = x - z
        u = u + rho*(r)   
        # check convergence
        if norm(r)/np.max([norm(x), norm(z)]) < 1e-14:
            is_converged = True
            break
    if not is_converged:
        print('Warning! Convergence criterion is not satisfied!')
    return x


In [4]:
from pyunlocbox import functions

norm_l1 = functions.norm_l1()

x0 = np.random.randn(5,)
tau = 0.7
print(prox_l1(x0, tau))
print(norm_l1.prox(x0, tau))

[-0.         -0.         -0.67691334  0.          0.        ]
[ 0.          0.         -0.67691334  0.          0.        ]


In [5]:
from sklearn.linear_model import Lasso

def lasso_sklearn(A, b, lmbda):
    """Solves
    0.5 / n_samples * ||A*x - y||^2_2 + alpha * ||x||_1,
    where lmbda = alpha*n_samples
    """
    alpha = lmbda/A.shape[0]
    rgr_lasso = Lasso(alpha=alpha, fit_intercept=False)
    rgr_lasso.fit(A, b)
    return rgr_lasso.coef_

In [6]:
np.random.seed(0)
A_ = np.random.randn(3,5)

def mv(v):
    return A_.dot(v)
def rmv(v):
    return A_.T.dot(v)

A = LinearOperator(A_.shape, matvec=mv, rmatvec=rmv)
b = np.random.randn(A_.shape[0],)

In [7]:
lmbda = 0.1
x_opt_1 = lasso_admm(A, b, lmbda)
print(x_opt_1)

[-7.60737356e-01 -4.07074992e-15 -5.95278300e-01  1.29171077e-15
  1.18008980e+00]


In [8]:
lmbda = 0.1
x_opt_2 = lasso_sklearn(A_, b, lmbda)
print(x_opt_2)

[-0.76062233 -0.         -0.59557565  0.          1.18016384]


### RPCA

Robust Principal Component Analysis

\begin{gather}
\min_{\mathbf{L}, \mathbf{S}} \lVert \mathbf{L} \rVert_* + \lambda \lVert \mathbf{S} \rVert_1  \\
\text{s.t. } \mathbf{X} = \mathbf{L} + \mathbf{S}
\end{gather}

https://statweb.stanford.edu/~candes/math301/Lectures/rpca.pdf

In [9]:
def prox_nuclear(X, tau):
    [U, S, V] = np.linalg.svd(X, full_matrices=False)
    S = np.maximum(S - tau, 0)
    S = np.diag(S)
    return U.dot(S).dot(V)

def RPCA_admm(X, lmbda, rho=0.1, max_iter=10000):
    """Solves RPCA via ADMM
    """
    M,N = X.shape
    normX = norm(X, 'fro')
    
    # initial solution
    L = np.zeros((M, N))
    S = np.zeros((M, N))
    U = np.zeros((M, N))
    
    is_converged = False
    for iteration in range(max_iter):          
        # update L and S
        L = prox_nuclear(X - S + (1/rho)*U, 1/rho)
        S = prox_l1(X - L + (1/rho)*U, lmbda/rho)
        # update U
        Z = X - L - S
        U = U + rho*Z
        # check convergence
        err = norm(Z, 'fro') / normX      
        if err < 1e-12:
            is_converged = True
            break
    if not is_converged:
        print('Warning! Convergence criterion is not satisfied!')
    return L, S

In [10]:
norm_nuc = functions.norm_nuclear()

X0 = np.random.randn(3,3)
tau = 1.3
print(prox_nuclear(X0, tau))
print(norm_nuc.prox(X0, tau)) # works only for square matrices

[[-0.21367621 -0.05831636  0.03830712]
 [-0.24143197 -0.06589144  0.04328308]
 [-0.40432885 -0.11034915  0.07248666]]
[[-0.21367621 -0.05831636  0.03830712]
 [-0.24143197 -0.06589144  0.04328308]
 [-0.40432885 -0.11034915  0.07248666]]


In [11]:
m = 4 
n = 3

np.random.seed(123)
base = 100 + np.cumsum(np.random.randn(m,n),axis=0)
scales = np.abs(np.random.randn(m,n))

L = np.dot(base, scales.T)
S = np.round(0.25 * np.random.randn(m,m))
X = L + S

In [12]:
#!pip install sporco

In [13]:
from sporco.admm import rpca

options = rpca.RobustPCA.Options({'Verbose': False, 'gEvalY': False,'MaxMainIter': 10000, 
                                  'RelStopTol': 1e-12, 'AutoRho': {'Enabled': True}})

lmbda = 0.4
rpca = rpca.RobustPCA(X, lmbda, options)
L_opt_1, S_opt_1 = rpca.solve()
print(L_opt_1)
print(S_opt_1)

[[253.97359246 485.05408746 212.26467636 359.88719381]
 [252.67146505 482.56720599 211.17639135 358.04204537]
 [251.35706571 480.05688684 210.07784977 356.17950728]
 [249.58862727 476.67941644 208.59983386 353.67358396]]
[[2.59751091 0.         0.         0.        ]
 [2.01670518 4.16760718 0.57015378 0.        ]
 [0.         7.44607564 0.         0.33214328]
 [0.         8.7423926  0.27574982 0.7993027 ]]


In [14]:
lmbda = 0.4
L_opt_2, S_opt_2 = RPCA_admm(X, lmbda)
print(L_opt_2)
print(S_opt_2)

[[253.97359246 485.05408746 212.26467636 359.88719381]
 [252.67146505 482.56720599 211.17639135 358.04204537]
 [251.35706571 480.05688685 210.07784978 356.17950728]
 [249.58862728 476.67941644 208.59983386 353.67358396]]
[[ 2.59751092  0.          0.          0.        ]
 [ 2.01670518  4.16760718  0.57015378 -0.        ]
 [ 0.          7.44607563 -0.          0.33214328]
 [-0.          8.7423926   0.27574982  0.7993027 ]]


### Linear programming

Solving standard form LP via ADMM

\begin{gather}
\min_{\mathbf{x}}  \mathbf{c}^T \mathbf{x}  \\
\text{s.t. } \mathbf{G}\mathbf{x} = \mathbf{h}, \mathbf{x} \geq \mathbf{0}  
\end{gather}

http://cermics.enpc.fr/~parmenta/frejus/Summerproject2018-4.pdf

https://github.com/nmchaves/admm-for-lp/blob/master/report/fall2016/report.pdf

In [15]:
dim = 5

np.random.seed(0)

c = np.random.randn(dim,)

G = np.random.randn(2,dim)
h = np.random.randn(2,)

A = -1*np.eye(dim)
b = np.zeros((dim,))

In [16]:
from cvxopt import matrix, solvers

# objective
c_ = matrix(c)
# inequalities
A_ = matrix(A)
b_ = matrix(b)
# equalities
G_ = matrix(G)
h_ = matrix(h)

sol=solvers.lp(c_, A_, b_, G_, h_, solver='glpk')
x_opt_1 = np.array(sol['x'])

print(x_opt_1)

[[-6.76992000e-17]
 [ 5.09006281e-01]
 [ 9.90548429e-01]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]


In [19]:
def lp_admm(c, G, h, rho=0.1, max_iter=10000):
    """Solves LP via ADMM
    """
    n,dim = G.shape
    x1 = lsqr(G, h)[0]
    x2 = x1.copy()
    u = np.zeros((n,))
    s = np.zeros((dim,))
    
    mv = lambda x: G.rmatvec(G.matvec(x)) + x
    GTGeye = LinearOperator((dim, dim), matvec=mv)
    GTh = G.rmatvec(h)
    
    is_converged = False
    for iteration in range(max_iter):          
        # update x1
        x1 = cg(GTGeye, x2 + GTh - 1./rho*(c + G.rmatvec(u) + s))[0]
        # update x2
        x2 = np.maximum(x1 + 1./rho*s, 0)
        # update u and s
        u = u + rho*(G.matvec(x1) - h)
        s = s + rho*(x1 - x2)
        # check convergence
        err = norm(x1 - x2)/np.max([norm(x1), norm(x2)])      
        if err < 1e-12:
            is_converged = True
            break
    if not is_converged:
        print('Warning! Convergence criterion is not satisfied!')  
    return x1

In [20]:
def mv(v):
    return G.dot(v)
def rmv(v):
    return G.T.dot(v)

G_op = LinearOperator(G.shape, matvec=mv, rmatvec=rmv)

x_opt_2 = lp_admm(c, G_op, h)
print(x_opt_2)

[ 2.87941457e-13  5.09006281e-01  9.90548429e-01  4.54997127e-14
 -5.04729366e-14]


### Set-constrained optimization

\begin{gather}
\min_{\mathbf{x}}  f(\mathbf{x})  \\
\text{s.t. } \mathbf{x} \in C
\end{gather}

\begin{gather}
\min_{\mathbf{x}, \mathbf{z}}  f(\mathbf{x}) + \mathbb{i}_C( \mathbf{z} )  \\
\text{s.t. } \mathbf{x} - \mathbf{z} =  \mathbf{0}
\end{gather}

$f(x) = \frac{1}{2} \| \mathbf{x} - \mathbf{x}_0 \|^2$

In [21]:
import sys
sys.path.append('./')
from utils.simplex_projection import euclidean_proj_simplex

def euclidean_proj_l1_ball(v, s=1):
    u = np.abs(v)
    if u.sum() <= s:
        return v
    w = euclidean_proj_simplex(u, s=s)
    w *= np.sign(v)
    return w

In [22]:
from scipy.optimize import minimize

def fmin_admm(f, grad, proj, x0, rho=0.2, max_iter=10000):
    
    x = x0.copy()
    z = x0.copy()
    u = x0.copy()
    
    is_converged = False
    for iteration in range(max_iter):          
        # update x
        func = lambda v: f(v) + 0.5*rho*norm(v - z + u)**2
        gradient = lambda v: grad(v) + rho*(v - z + u)
        x = minimize(func, x, method='BFGS', jac=gradient, options={'maxiter': 100, 'disp': False}, tol=1e-12).x
        # update z
        z = proj(x + u)
        # update u
        u = u + x - z
        # check convergence
        err = norm(x - z)/np.max([norm(x), norm(z)])
        if err < 1e-12:
            is_converged = True
            break
    if not is_converged:
        print('Warning! Convergence criterion is not satisfied!')  
    return x


In [23]:
dim = 4

np.random.seed(10)
x0 = np.random.randn(dim,)

x_opt_1 = euclidean_proj_l1_ball(x0, 0.1)
print(x_opt_1)

[ 0.   0.  -0.1 -0. ]


In [24]:
projection = lambda x: euclidean_proj_l1_ball(x, 0.1)

x_opt_2 = fmin_admm(lambda x: 0.5*norm(x - x0)**2, lambda x: x - x0, projection, x0)
print(x_opt_2)

[ 1.18922531e-14  6.57678355e-15 -1.00000000e-01 -7.40251142e-17]


### Consensus optimization

Suppose we have a convex optimization problem with $N$ terms in the objective

\begin{array}{ll} \mbox{minimize} & \sum_{i=1}^N f_i(x)\\
\end{array}

For example, we might be fitting a model to data and $f_i$ is the loss function for the $i$th block of training data.

We can convert this problem into consensus form

\begin{array}{ll} \mbox{minimize} & \sum_{i=1}^N f_i(x_i)\\
\mbox{subject to} & x_i = z
\end{array}

We interpret the $x_i$ as local variables, since they are particular to a given $f_i$. The variable $z$, by contrast, is global. The constraints $x_i = z$ enforce consistency, or consensus.

We can solve a problem in consensus form using the Alternating Direction Method of Multipliers (ADMM). Each iteration of ADMM reduces to the following updates:

\begin{array}{lll}
% xbar, u parameters in prox.
% called proximal operator.
x^{k+1}_i & := & \mathop{\rm argmin}_{x_i}\left(f_i(x_i) + \frac{\rho}{2}\left\|x_i - \overline{x}^k + u^k_i \right\|^2_2 \right) \\
% u running sum of errors.
u^{k+1}_i & := & u^{k}_i + x^{k+1}_i - \overline{x}^{k+1}
\end{array}

where $\overline{x}^k =  \frac{1}{N}\sum_{i=1}^N x^k_i$.

http://proceedings.mlr.press/v32/zhange14.pdf

In [25]:
from multiprocessing import Process, Pipe

def consensus_admm(f_list, g_list, x0, rho=0.1, max_iter=1000):

    N = len(f_list)

    def run_worker(f, grad, pipe):
        xbar = x0.copy()
        u = x0.copy()
        x = x0.copy()
        while True:
            func = lambda v: f(v) + 0.5*rho*norm(v - xbar + u)**2
            gradient = lambda v: grad(v) + rho*(v - xbar + u)
            x = minimize(func, x, method='BFGS', jac=gradient, options={'maxiter': 100, 'disp': False}, tol=1e-12).x
            pipe.send(x)
            xbar = pipe.recv()
            u = u + x - xbar

    # Setup the workers.
    pipes = []
    procs = []
    for i in range(N):
        local, remote = Pipe()
        pipes += [local]
        procs += [Process(target=run_worker, args=(f_list[i], g_list[i], remote))]
        procs[-1].start()

    is_converged = False
    for iteration in range(max_iter):
        # Gather and average xi
        x_list = [pipe.recv() for pipe in pipes]
        xbar = sum(x_list)/N
        # Scatter xbar
        for pipe in pipes:
            pipe.send(xbar)
        # check convergence
        if np.all([np.allclose(xbar, xi, rtol=1e-8) for xi in x_list]):
            is_converged = True
            break
            
    if not is_converged:
        print('Warning! Convergence criterion is not satisfied!') 

    [p.terminate() for p in procs]

    return xbar

In [26]:
np.random.seed(0)
A = np.random.randn(5,4)
b = np.random.randn(5,)

x0 = np.random.randn(4,)

f1 = lambda x: 0.123*np.sum(x**2)/2
f2 = lambda x: np.sum((np.dot(A,x) - b)**2)/2

g1 = lambda x: 0.123*x
g2 = lambda x: A.T.dot(np.dot(A,x) - b)


In [27]:
x_opt_1 = minimize(lambda x: f1(x) + f2(x), x0, method='BFGS', 
                   jac=lambda x: g1(x) + g2(x), 
                   options={'maxiter': 100, 'disp': False}, tol=1e-12).x

print(x_opt_1)

[ 0.6074129  -0.30633395 -1.21027275 -0.6287909 ]


In [28]:
f_list = [f1, f2]
g_list = [g1, g2]

x_opt_2 = consensus_admm(f_list, g_list, x0)
print(x_opt_1)

[ 0.6074129  -0.30633395 -1.21027275 -0.6287909 ]
